In [ ]:
# score 계산하기

# # score 칼럼 구하기
# score_df['score'] = score_df['category_sum'].astype('int') + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# # score_df[['주변시설의총합','score']]
# # sorted_score_df =score_df[['정류장번호','정류장명','위도','경도','도로명주소','시','구','읍/면/동','리','score']].sort_values(by='score',ascending=False)
# # sorted_score_df
# score_df.head(2)


In [1]:
>## 6.1 최적의 클러스터 갯수 구하기

Elbow point와 silhouette_score 를 통해, 좌표를 통한 클러스터를 묶을 클러스터 갯수를 구하기 위해, 

다음 2가지 방법으로 최적의 클러스터 갯수를 구한다.

SyntaxError: invalid decimal literal (2638033412.py, line 5)

In [ ]:
>### 6.1.1 Elbow point


# 클러스터링할 데이터 넣기
data = selected_rows.copy()





In [ ]:

coordinates = data[['위도', '경도']]
num_clusters_range = range(2, 10)  # You can change this range

inertias = []
for n in num_clusters_range:
    kmeans = KMeans(n_clusters=n, random_state=98)

    # Fit the model to the coordinates
    kmeans.fit(coordinates)
    inertias.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(num_clusters_range, inertias, marker='o')
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()


# # Add a new column to your DataFrame with the cluster labels
# data['cluster'] = kmeans.labels_

# # Print the cluster centers (the centroids)

# '''
# 센트럴 포인트에서 가까운 정류장만 고려하기?
# '''
# print("Cluster Centers:")
# print(kmeans.cluster_centers_)
# print(data['cluster'].value_counts())

# # Print the first few rows of the DataFrame with the cluster labels
# data.head()



In [ ]:
> ### 6.1.2 실루엣 스코어 

In [ ]:


coordinates = selected_rows[['위도', '경도']]
num_clusters_range = range(2, 10)  # You can change this range

inertias = []
silhouette_scores = []

for n in num_clusters_range:
    kmeans = KMeans(n_clusters=n, random_state=98)

    # Fit the model to the coordinates
    kmeans.fit(coordinates)
    inertias.append(kmeans.inertia_)

    # Calculate silhouette score
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(coordinates, labels)
    silhouette_scores.append(silhouette_avg)

# Plot the elbow curve
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(num_clusters_range, inertias, marker='o')
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')

# Plot the silhouette scores
plt.subplot(1, 2, 2)
plt.plot(num_clusters_range, silhouette_scores, marker='o', color='r')
plt.title('Silhouette Score')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')

plt.tight_layout()
plt.show()


In [ ]:
'''
빨간선은 전체 실루엣 계수의 평균,

이 빨간선과, 각 그룹의 평균 실루엣 계수가 유사하거나, 
해당 갯수로한 클러스터의 실루엣 계수가 높아야 군집화가 잘 되었다고 판단할 수 있다.
'''
def visualize_silhouette(cluster_lists, X_features): 

    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    import numpy as np

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(10*n_cols, 20), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")
visualize_silhouette([2,3,4,5,6,7,8], coordinates)


In [2]:
________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

NameError: name '________________________________________________________________________________________________________________________________________________________________________________________________________' is not defined

In [ ]:
# scaled_data_cp = scaled_data.copy()


# scaled_data_cp.sort_values(by='minmax_sum_score',inplace=True,ascending=False)
# # scaled_data_cp.drop_duplicates(subset='정류장명',keep='first',inplace=True)

# selected = ['부성동','불당동','쌍용동','청룡동','성정동','신방동','신안동','백성동']
# selected_scaled_data = scaled_data_cp[scaled_data_cp['대분류'].isin(selected)]

# selected_scaled_data.sort_values(by='minmax_sum_score',inplace=True,ascending=False)
# selected_scaled_data.head()
# print(len(selected_scaled_data))

In [ ]:
# data['minmax_sum_score'].describe()
# import numpy as np

# # Assuming data['minmax_sum_score'] contains your data
# # Calculate quartiles
# first_quartile = np.percentile(data['minmax_sum_score'], 25)
# second_quartile = np.percentile(data['minmax_sum_score'], 50)
# third_quartile = np.percentile(data['minmax_sum_score'], 75)

# # Create 'quartile classification' column based on quartile values
# data['quartile classification'] = np.select(
#     [
#         data['minmax_sum_score'] <= first_quartile,
#         data['minmax_sum_score'] <= second_quartile,
#         data['minmax_sum_score'] <= third_quartile,
#         data['minmax_sum_score'] > third_quartile,
#     ],
#     [1, 2, 3, 4],
#     default=0  # This is the default value if none of the conditions are met
# )
# data['quartile classification'] = data['quartile classification']*4

In [ ]:


# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(
    data_cp,
    lat='위도',
    lon='경도',
    color='blue',  # Set marker color to blue
    size_max=20,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# Create larger red square markers for '위도' and '경도' from the 'park' DataFrame
park_trace = go.Scattermapbox(
    lat=park['위도'],
    lon=park['경도'],
    mode='markers',
    marker=dict(size=5, color='red', symbol='circle'),  # Increase the size to 15
    hovertext=park['공원명']
)

# Add the park_trace to the figure
fig.add_trace(park_trace)

# Create yellow markers for '위도' and '경도' from the 'ground_truth' DataFrame
ground_truth_trace = go.Scattermapbox(
    lat=ground_truth['위도'],
    lon=ground_truth['경도'],
    mode='markers',
    marker=dict(size=20, color='pink', symbol='circle'),
    hovertext=ground_truth['장소명']  # Replace with the appropriate column from ground_truth
)

# Add the ground_truth_trace to the figure
fig.add_trace(ground_truth_trace)

cluster_centers_df_trace = go.Scattermapbox(
    lat = cluster_centers_df['위도'],
    lon = cluster_centers_df['경도'],
    mode='markers',
    marker = dict(size=15,color='black',symbol='circle'),
    hovertext=cluster_centers_df['cluster_label']
)

fig.add_trace(cluster_centers_df_trace)

# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

# Show the plot
# fig.show()


In [1]:
# import plotly.graph_objects as go
# import plotly.express as px

# # Create a scatter map using Plotly Express
# fig = px.scatter_mapbox(
#     data_cp,
#     lat='위도',
#     lon='경도',
#     color='cluster',  # Color points based on the quartile classification column
#     color_continuous_scale='blues',   # Choose a blue gradient color scale
#     range_color=[1, 4],               # Set the range of color scale based on your quartile values
#     size_max=20,      # Adjust point size
#     zoom=9,           # Initial map zoom level
#     hover_name='정류장명',
#     size='for_marker_size'
# )

# # Create larger red square markers for '위도' and '경도' from the 'park' DataFrame
# park_trace = go.Scattermapbox(
#     lat=park['위도'],
#     lon=park['경도'],
#     mode='markers',
#     marker=dict(size=5, color='red', symbol='circle'),  # Increase the size to 15
#     hovertext=park['공원명']
# )

# # Add the park_trace to the figure
# fig.add_trace(park_trace)

# # Customize the map layout
# map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
# fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
# fig.update_layout(
#     margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
#     mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
# )

# # Show the plot
# fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly.express as px

# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(
    data_cp,
    lat='위도',
    lon='경도',
    color='cluster',  # Color points based on the quartile classification column
    color_continuous_scale='blues',   # Choose a blue gradient color scale
    range_color=[1, 1],               # Set the range of color scale based on your quartile values
    size_max=20,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# Create larger red square markers for '위도' and '경도' from the 'park' DataFrame
park_trace = go.Scattermapbox(
    lat=park['위도'],
    lon=park['경도'],
    mode='markers',
    marker=dict(size=5, color='red', symbol='circle'),  # Increase the size to 15
    hovertext=park['공원명']
)

# Add the park_trace to the figure
fig.add_trace(park_trace)

# Create yellow markers for '위도' and '경도' from the 'ground_truth' DataFrame
ground_truth_trace = go.Scattermapbox(
    lat=ground_truth['위도'],
    lon=ground_truth['경도'],
    mode='markers',
    marker=dict(size=20, color='yellow', symbol='circle'),
    hovertext=ground_truth['장소명']  # Replace with the appropriate column from ground_truth
)

# Add the ground_truth_trace to the figure
fig.add_trace(ground_truth_trace)

# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

# Show the plot
fig.show()


In [ ]:

# 주거지, 회사 좌표 추가

import plotly.graph_objects as go
import plotly.express as px

# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(
    data_cp,
    lat='위도',
    lon='경도',
    color='cluster',  # Color points based on the quartile classification column
    color_continuous_scale='blues',   # Choose a blue gradient color scale
    range_color=[1, 4],               # Set the range of color scale based on your quartile values
    size_max=20,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# Create larger red square markers for '위도' and '경도' from the 'park' DataFrame
park_trace = go.Scattermapbox(
    lat=park['위도'],
    lon=park['경도'],
    mode='markers',
    marker=dict(size=5, color='red', symbol='circle'),  # Increase the size to 15
    hovertext=park['공원명']
)

# Add the park_trace to the figure
fig.add_trace(park_trace)

# Create yellow markers for '위도' and '경도' from the 'ground_truth' DataFrame
ground_truth_trace = go.Scattermapbox(
    lat=ground_truth['위도'],
    lon=ground_truth['경도'],
    mode='markers',
    marker=dict(size=20, color='yellow', symbol='circle'),
    hovertext=ground_truth['장소명']  # Replace with the appropriate column from ground_truth
)

# Add the ground_truth_trace to the figure
fig.add_trace(ground_truth_trace)

# Create green markers for '위도' and '경도' from the 'houseAndcmp' DataFrame
house_and_cmp_trace = go.Scattermapbox(
    lat=houseAndcmp['위도'],
    lon=houseAndcmp['경도'],
    mode='markers',
    marker=dict(size=5, color='green', symbol='circle'),
    hovertext=houseAndcmp['이름']  # Replace with the appropriate column from houseAndcmp
)

# Add the house_and_cmp_trace to the figure
fig.add_trace(house_and_cmp_trace)

# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

# Show the plot
fig.show()


## folium


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a Folium map centered at the specified latitude and longitude
m = folium.Map(location=[36.83682, 127.2077], zoom_start=9)

# Create a marker cluster for the bus stops
marker_cluster = MarkerCluster()

# Add bus stop markers to the marker cluster
for idx, row in data_cp.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=None,  # You can customize the icon if needed
        popup=row['정류장명'],
    ).add_to(marker_cluster)

# Add the marker cluster to the map
marker_cluster.add_to(m)

# Add red circle markers for parks
for idx, row in park.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        popup=row['공원명']
    ).add_to(m)

# Display the map
m


In [ ]:
import plotly.express as px
import folium
from folium.plugins import MarkerCluster

# Assuming df, bus_station_Main, map_style, and fig are defined as in your provided code

# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(
    data_cp,
    lat='위도',
    lon='경도',
    color='cluster',
    size_max=10,
    zoom=9,
    hover_name='정류장명',
    size='for_marker_size'
)

# Create a new scatter trace for red square markers
red_markers = px.scatter_mapbox(
    bus_station_Main,
    lat='위도',
    lon='경도',
    color_discrete_sequence=['red'],
    hover_name='정류장명',
    size='default_marker_size'
)
red_markers.update_traces(marker_symbol='square')

# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner", "stamen-watercolor"]
fig.update_layout(mapbox_style=map_style[0])
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),
)

# Convert Plotly figure to HTML
fig_html = fig.to_html()

# Create a Folium map
m = folium.Map(location=[36.83682, 127.2077], zoom_start=9, tiles='OpenStreetMap')

# Create a MarkerCluster for red square markers
marker_cluster = MarkerCluster().add_to(m)

# Add red_markers to the MarkerCluster
for _, row in bus_station_Main.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=10,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        popup=row['정류장명']
    ).add_to(marker_cluster)

# Add Plotly Express figure HTML to the Folium map
folium_map = folium.IFrame(html=fig_html, width='100%', height='400px')
popup = folium.Popup(folium_map, max_width=1000)
popup.add_to(m)

# # Display the Folium map
# m.save('folium_map.html')  # Save the map as an HTML file
m


In [ ]:
current_datetime = datetime.now()
when = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')
when_sanitized = when.replace(':', '-')


path = r'E:\제출용데이터dir\folium시각화\{when}.html'.format(when=when)
m.save(path)